<a href="https://colab.research.google.com/github/navneetkrc/Colab_fastai/blob/master/Deduplicate_Based_on_address.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://towardsdatascience.com/de-duplicate-the-duplicate-records-from-scratch-f6e5ad9e79da

In [0]:
!pip install sparse-dot-topn

In [0]:
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/susanli2016/NLP-with-Python/master/data/Seattle_Hotels_Duplicates.csv', encoding="latin-1")

In [19]:
df.tail(3)

,name,address,name_address
165,Pike's Place Lux Suites by Barsala,"2nd Ave and Stewart St, Seattle, WA 98101",Pike's Place Lux Suites by Barsala 2nd Ave and...
166,Pike's Place Lux Suites by Barsala,"2rd Ave and Stewart St, Seattle, WA 98101",Pike's Place Lux Suites by Barsala 2rd Ave and...
167,citizenM Seattle South Lake Union hotel,"201 Westlake Ave N, Seattle, WA 98109",citizenM Seattle South Lake Union hotel 201 We...


In [10]:
df.name.value_counts()

Renaissance Seattle Hotel              2
Roy Street Commons                     2
Pike's Place Lux Suites by Barsala     2
Ace Hotel Seattle                      2
Quality Inn & Suites Seattle Center    2
                                      ..
The Inn at El Gaucho                   1
Vermont Inn                            1
Hyatt House Seattle                    1
Kings Inn                              1
Stay Alfred on 4th Avenue              1
Name: name, Length: 155, dtype: int64

In [11]:
df.loc[df['name'] == 'Roy Street Commons']

,name,address
82,Roy Street Commons,"621 12th Ave E, Seattle, WA 98102"
90,Roy Street Commons,"621 12th Avenue East, Seattle, Washington 98102"


In [0]:
df['name_address'] = df['name'] + ' ' + df['address']
name_address = df['name_address']
vectorizer = TfidfVectorizer("char", ngram_range=(1, 4), sublinear_tf=True)
tf_idf_matrix = vectorizer.fit_transform(name_address)

In [13]:
tf_idf_matrix

<168x3218 sparse matrix of type '<class 'numpy.float64'>'
	with 6002 stored elements in Compressed Sparse Row format>

In [0]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
  
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 5)

In [15]:
matches

<168x168 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [0]:
def get_matches_df(sparse_matrix, name_vector, top=840):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})

matches_df = get_matches_df(matches, name_address)

In [17]:
matches_df[matches_df['similarity'] < 0.99999].sort_values(by=['similarity'], ascending=False).head(30)

,left_side,right_side,similarity
826,Pike's Place Lux Suites by Barsala 2nd Ave and...,Pike's Place Lux Suites by Barsala 2rd Ave and...,0.715406
831,Pike's Place Lux Suites by Barsala 2rd Ave and...,Pike's Place Lux Suites by Barsala 2nd Ave and...,0.715406
206,Holiday Inn Express & Suites Seattle-City Cent...,Holiday Inn Express & Suites Seattle City Cent...,0.712321
256,Holiday Inn Express & Suites Seattle City Cent...,Holiday Inn Express & Suites Seattle-City Cent...,0.712321
181,Travelodge Seattle by The Space Needle 200 6th...,Travelodge Seattle by The Space Needle 200 6th...,0.669974
211,Travelodge Seattle by The Space Needle 200 6th...,Travelodge Seattle by The Space Needle 200 6th...,0.669974
791,citizenM Seattle South Lake Union hotel 201 We...,citizenM Seattle South Lake Union hotel 201 We...,0.651961
836,citizenM Seattle South Lake Union hotel 201 We...,citizenM Seattle South Lake Union hotel 201 We...,0.651961
586,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.627400
551,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.627400


In [18]:
matches_df[matches_df['similarity'] < 0.50].right_side.nunique()

152